In [1]:
# Import Packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import random
import numpy as np
import pandas as pd
from collections import Counter
from datetime import date, timedelta


In [2]:
# Declare Variables
active_indicator_types = [0,1]
day_one_prob = (70,30)
distinct_users = 10000
start_id = 10001
end_id = start_id + distinct_users
start_date = '1/1/2021'
end_date = '3/31/2021'
start_date_v2 = date(2021,1,1)
end_date_v2 = date(2021,3,31)
period_days = (end_date_v2-start_date_v2).days+1
output_start_date = '2/1/2021'

In [3]:
# Randomly generate users activity for the first day of 2021-01-01, 30% chance of being active.
# This will change with every run
random_activity_generator = random.choices(active_indicator_types,weights=day_one_prob,k=distinct_users)

In [4]:
zeros_generated = Counter(random_activity_generator)[0]
ones_generated = Counter(random_activity_generator)[1]

# print('Percentage of 0s = '+str(round(zeros_generated*100/(zeros_generated+ones_generated),2))+
#       ' and percentage of 1s = '+str(round(ones_generated*100/(zeros_generated+ones_generated),2)))

In [5]:
# Generate Date_key
date_range = pd.date_range(start=start_date, end=end_date)
date_df = pd.DataFrame(np.repeat(date_range,distinct_users),columns=['Date_key'])
date_df.head()

,Date_key
0,2021-01-01
1,2021-01-01
2,2021-01-01
3,2021-01-01
4,2021-01-01


In [6]:
# Generate UID
user_id_range = pd.Series(range(start_id,end_id))
user_id_df = pd.DataFrame(np.tile(user_id_range, period_days),columns=['UID'])
user_id_df.head()

,UID
0,10001
1,10002
2,10003
3,10004
4,10005


In [7]:
# Generate Active_indicator
active_indicator = random_activity_generator + list(np.tile(0, (period_days-1)*distinct_users))
active_indicator_df = pd.DataFrame(active_indicator,columns = ['Active_indicator'])
active_indicator_df.head()

,Active_indicator
0,0
1,0
2,1
3,0
4,0


In [8]:
# Combine Date_key, UID and Active_indicator into one DF
input_data = pd.concat([date_df, user_id_df, active_indicator_df],axis=1)
input_data.head(5)

,Date_key,UID,Active_indicator
0,2021-01-01,10001,0
1,2021-01-01,10002,0
2,2021-01-01,10003,1
3,2021-01-01,10004,0
4,2021-01-01,10005,0


In [9]:
input_data.tail(5)

,Date_key,UID,Active_indicator
899995,2021-03-31,19996,0
899996,2021-03-31,19997,0
899997,2021-03-31,19998,0
899998,2021-03-31,19999,0
899999,2021-03-31,20000,0


In [10]:
# Function to generate Active_indicator randomly for all the days after the 2021-01-01
# It returns 0 or 1 depending on the probabality of the activity for the subsequent days
def generate_random_number(activity_type):
    current_active_prob_tmr_active = 97.27
    current_active_prob_tmr_inactive = 100 - current_active_prob_tmr_active
    
    current_inactive_prob_tmr_active = 1.15
    current_inactive_prob_tmr_inactive = 100 - current_active_prob_tmr_active
    
    if activity_type == 'active':
        return random.choices(active_indicator_types,weights=
                              (current_active_prob_tmr_inactive,current_active_prob_tmr_active))
    else:
        return random.choices(active_indicator_types,weights=
                              (current_inactive_prob_tmr_inactive,current_inactive_prob_tmr_active))
    

In [11]:
# Fill in Active_indicator values for all the days following 2021-01-01
date_counter = start_date_v2 + timedelta(days=1)

while date_counter <= end_date_v2:
    uid_counter = start_id
    print(date_counter)
    
    while uid_counter < end_id:        
        input_data.loc[(input_data.Date_key==date_counter)&
                       (input_data.UID==uid_counter),'Active_indicator'] =  np.where(input_data[(input_data.Date_key==(date_counter - timedelta(days=1)))&(input_data.UID==uid_counter)]['Active_indicator']> 1, generate_random_number('active'), generate_random_number('inactive'))
        uid_counter+=1
        
    date_counter = date_counter + timedelta(days=1)

In [12]:
input_data.to_csv('InputData.csv')

In [13]:
input_data = pd.read_csv('InputData.csv')

In [14]:
input_data.drop(columns=['Unnamed: 0'],inplace=True)
input_data['Date_key'] = input_data['Date_key'].astype('datetime64')

In [15]:
DAU_Count, MAU_Count, Reengaged_count, New_Inactive_count = [], [], [], []

for date_val in pd.date_range(start=output_start_date, end=end_date):
    
    # DAU_Count: number of users who are active on the date
    dau_count = input_data[input_data.Date_key==date_val]['Active_indicator'].sum()
    DAU_Count.append(dau_count)

    # MAU_Count: number of users who are active on ANY of the previous 30 days, including the current date
    mau_grouped_df = input_data[(input_data.Date_key>=(date_val-timedelta(days=29)))&
                                (input_data.Date_key<=date_val)][['UID','Active_indicator']].groupby('UID').sum()
    mau_grouped_df['UID'] = mau_grouped_df.index
    mau_grouped_df.reset_index(drop=True,inplace=True)
    mau_grouped_df['active_unique_uid']=0
    mau_grouped_df.loc[mau_grouped_df['Active_indicator']>0,'active_unique_uid']=1
    mau_count = mau_grouped_df['active_unique_uid'].sum()
    MAU_Count.append(mau_count)
    
    # Reengaged_count: number of users who were not MAU (i.e., not 30-day active) yesterday, but became active today
    yesterday_mau_df = input_data[(input_data.Date_key>=((date_val-timedelta(days=1))-timedelta(days=29)))&
                              (input_data.Date_key<=(date_val-timedelta(days=1)))][['UID','Active_indicator']].groupby('UID').sum()
    yesterday_mau_df['yesterday_mau_active_unique_uid']=0
    yesterday_mau_df.loc[yesterday_mau_df['Active_indicator']>0,'yesterday_mau_active_unique_uid']=1
    yesterday_mau_df.drop(columns=['Active_indicator'],inplace=True)
    yesterday_mau_df['UID'] = yesterday_mau_df.index
    yesterday_mau_df.reset_index(drop=True,inplace=True)
    today_df = input_data[input_data.Date_key==date_val]
    today_and_yesterday_mau_combined_df = yesterday_mau_df.merge(today_df,left_on='UID',right_on='UID')
    reengaged_count = len(today_and_yesterday_mau_combined_df[
                          (today_and_yesterday_mau_combined_df['Active_indicator']==1)&
                          (today_and_yesterday_mau_combined_df['yesterday_mau_active_unique_uid']==0)]['UID'])
    Reengaged_count.append(reengaged_count)
    
    #New_Inactive_count: number of users who were MAU (i.e., 30-day active) yesterday, but became 30- day inactive today (haven’t been active at all in the past 30 days)
    today_mau_df = input_data[(input_data.Date_key>=(date_val-timedelta(days=29)))&
                              (input_data.Date_key<=date_val)][['UID','Active_indicator']].groupby('UID').sum()
    today_mau_df['today_mau_active_unique_uid']=0
    today_mau_df.loc[today_mau_df['Active_indicator']>0,'today_mau_active_unique_uid']=1
    today_mau_df.drop(columns=['Active_indicator'],inplace=True)
    today_mau_df['UID'] = today_mau_df.index
    today_mau_df.reset_index(drop=True,inplace=True)
    today_mau_and_yesterday_mau_combined_df = yesterday_mau_df.merge(today_mau_df,left_on='UID',right_on='UID')
    new_inactive_count = len(today_mau_and_yesterday_mau_combined_df[
                            (today_mau_and_yesterday_mau_combined_df['yesterday_mau_active_unique_uid']==1)&
                            (today_mau_and_yesterday_mau_combined_df['today_mau_active_unique_uid']==0)]['UID'])
    New_Inactive_count.append(new_inactive_count)
    

In [16]:
output_df = pd.DataFrame({'Date_key':pd.date_range(start=output_start_date, end=end_date),
                         'DAU_Count':DAU_Count,
                         'MAU_Count':MAU_Count,
                         'Reengaged_count':Reengaged_count,
                         'New_Inactive_count':New_Inactive_count})

In [17]:
output_df

,Date_key,DAU_Count,MAU_Count,Reengaged_count,New_Inactive_count
0,2021-02-01,2915,10000,0,0
1,2021-02-02,3031,10000,0,0
2,2021-02-03,3025,10000,0,0
3,2021-02-04,2915,10000,0,0
4,2021-02-05,3064,10000,0,0
5,2021-02-06,2984,10000,0,0
6,2021-02-07,2983,9999,0,1
7,2021-02-08,2985,10000,1,0
8,2021-02-09,2986,10000,0,0
9,2021-02-10,3022,10000,0,0


In [18]:
output_df.to_csv('OutputData.csv')